In [4]:
import sqlite3


In [1]:
pip install flask transformers torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import sqlite3
from flask import Flask, request, render_template_string
from transformers import pipeline
from werkzeug.serving import run_simple

C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [7]:
def init_db():
    conn = sqlite3.connect('medical_diagnosis.db')
    conn.execute('''
        CREATE TABLE IF NOT EXISTS diagnoses (
            id INTEGER PRIMARY KEY,
            symptoms TEXT,
            diagnosis TEXT
        )
    ''')
    conn.close()

init_db()


In [8]:
symptom_diagnosis = {
    'fever': 'Flu or Viral Infection',
    'cough': 'Cold or Respiratory Infection',
    'headache': 'Migraine or Stress',
    'fatigue': 'Anaemia or Chronic Fatigue Syndrome',
    'vomiting': 'Food Poisoning',
    'breathlessness': 'Pneumonia or Asthma',
    'loss of taste': 'COVID-19',
    'sore throat': 'Strep Throat or Tonsillitis',
    'joint pain': 'Arthritis or Gout'
}

def get_diagnosis(symptoms):
    symptoms = symptoms.lower()
    for symptom, diagnosis in symptom_diagnosis.items():
        if symptom in symptoms:
            return diagnosis
    return 'Consult a medical professional for detailed analysis.'


In [9]:
medical_nlp = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def get_diagnosis_nlp(symptoms):
    possible_conditions = [
        "Flu", "Cold", "Migraine", "Food Poisoning", "Anaemia",
        "Chronic Fatigue", "Pneumonia", "COVID-19", "Tonsillitis",
        "Arthritis"
    ]
    diagnosis_result = medical_nlp(symptoms, candidate_labels=possible_conditions)
    return f"Based on symptoms, the most probable condition is: {diagnosis_result['labels'][0]}"


In [10]:
app = Flask(__name__)


In [11]:
@app.route('/')
def home():
    html_content = """
    <!DOCTYPE html>
    <html>
    <head><title>Medical Diagnosis Bot</title></head>
    <body>
        <h1>Medical Diagnosis Bot</h1>
        <form action="/diagnose" method="post">
            <input type="text" name="symptoms" placeholder="Enter your symptoms" required>
            <button type="submit">Diagnose</button>
        </form>
        <br>
        <a href="/history">View Diagnosis History</a>
    </body>
    </html>
    """
    return render_template_string(html_content)


In [12]:
@app.route('/diagnose', methods=['POST'])
def diagnose():
    symptoms = request.form['symptoms']
    rule_based_diagnosis = get_diagnosis(symptoms)
    nlp_based_diagnosis = get_diagnosis_nlp(symptoms)

    conn = sqlite3.connect('medical_diagnosis.db')
    conn.execute('INSERT INTO diagnoses (symptoms, diagnosis) VALUES (?, ?)', (symptoms, nlp_based_diagnosis))
    conn.commit()
    conn.close()

    return f"""
    <h2>Diagnosis Results</h2>
    Symptoms: {symptoms}<br><br>
    <b>Rule-Based Diagnosis:</b> {rule_based_diagnosis}<br>
    <b>NLP-Based Diagnosis:</b> {nlp_based_diagnosis}<br><br>
    <a href="/">Back to Home</a> | <a href="/history">View History</a>
    """


In [13]:
@app.route('/history', methods=['GET'])
def history():
    conn = sqlite3.connect('medical_diagnosis.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM diagnoses ORDER BY id DESC")
    records = cursor.fetchall()
    conn.close()

    html_content = "<h1>Diagnosis History</h1><ul>"
    for row in records:
        html_content += f"<li><b>ID:</b> {row[0]} | <b>Symptoms:</b> {row[1]} | <b>Diagnosis:</b> {row[2]}</li>"
    html_content += "</ul><br><a href='/'>Back to Home</a>"

    return html_content


In [16]:
if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Oct/2025 07:00:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 07:00:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2025 07:00:39] "POST /diagnose HTTP/1.1" 200 -


In [ ]:
from werkzeug.serving import run_simple

if __name__ == '__main__':
    run_simple('localhost', 5000, app, use_reloader=False)


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Oct/2025 07:01:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 07:01:15] "POST /diagnose HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 07:02:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 07:02:10] "POST /diagnose HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 07:02:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 07:02:27] "POST /diagnose HTTP/1.1" 200 -
